First lets import the necessary libraries

In [ ]:
import os
import gradio as gr # for the chat interface
import boto3 # to read the files in the S3 Object Storage
from PyPDF2 import PdfReader # to open the pdfs and extract the texts
import requests # to do the API calls
import psycopg2 # to connect to postgreSQL
from psycopg2.extras import execute_values 
from psycopg2 import OperationalError
from markdownify import markdownify as md # to open and split the markwdown files
import time # for the wait function

Then we set the variables and credentials needed

In [ ]:
# connection info for the S3 bucket which stores the documents to ingest
S3_endpoint = "https://s3.gra.io.cloud.ovh.net/" 
S3_container_name = "rag-knowledge-files"
S3_secret_key = "yourS3secretkey"
S3_access_key = "yourS3accesskey"

# connection info for PostgreSQL database with PGVector extension to store the embedddings
postgre_host = 'xxxx.database.cloud.ovh.net'
postgre_db = 'defaultdb'
postgre_port = '20184'
sslmode = "require"
postgre_password = "xxxx"
postgre_user = "xxxx"

# connection info for AI endpoint models (LLM and Embedding)
ovhcloud_embedding_endpoint_url = 'https://multilingual-e5-base.endpoints.kepler.ai.cloud.ovh.net/api/text2vec'
ovhcloud_llm_endpoint_url = 'https://mixtral-8x22b-instruct-v01.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1/chat/completions'
ovhcloud_ai_endpoint_api_key = "xxxx"


Lets connect to the postgreSQL DB and initiate the vector database

In [ ]:
# Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname="defaultdb",
        user=postgre_user,
        password=postgre_password,
        host=postgre_host,
        port=postgre_port,
        sslmode=sslmode
    )
    print("Connection to PostgreSQL DB successful")
except OperationalError as e:
    print(f"The error '{e}' occurred")

In [ ]:
cur = conn.cursor()
# check if the pgvector extension exists, if not activate it. Without this Postgre will now accept vectors and you won’t be able to do vector similarity search
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
conn.commit()

# check if the embeddings table exists, if not create it
# embeddings table stores the embeddings but also the document name, the page number and the text itself
cur.execute("CREATE TABLE IF NOT EXISTS embeddings (id SERIAL PRIMARY KEY, document_name TEXT, page_number INT, text TEXT, embedding VECTOR)")
conn.commit()
# we make sure the table is empty. This means every time we launch this job it will regenerate the full tabled based on all the documents in the object storage.
cur.execute("TRUNCATE TABLE embeddings;")
conn.commit()

The first dataset we will ingest in the knowledge base is list of PDF files containing the english "Terms & Conditions" for OVHcloud services. Those can be downloaded here https://www.ovhcloud.com/en-ie/terms-and-conditions/contracts/.
The documents have been uploaded into a S3 Object Storage bucket as PDF so we will connect to the S3 using boto3 (we could have mounted the S3 bucket in the notebook but this is also another way).

In [ ]:
# Connect to S3 and print the list of files
s3 = boto3.client('s3',
                  endpoint_url=S3_endpoint,
                  aws_access_key_id=S3_access_key,
                  aws_secret_access_key=S3_secret_key)
bucket = s3.list_objects(Bucket=S3_container_name)

Now for each pdf in the bucket we will extract the text from each page. The text from each page will be sent to the embedding model to calculate the embedding vector. The vector is then inserted in the PGVector table including the text itself and some metadata.

In [ ]:
# only select the pdf files and then for each count the number of pages and display the nb
for obj in bucket['Contents']:
    if obj['Key'].endswith('.pdf'):
        # Download the PDF file
        s3.download_file(S3_container_name, obj['Key'], 'temp.pdf')

        # Read the PDF file
        with open('temp.pdf', 'rb') as file:
            reader = PdfReader(file)
            print(f"Number of pages in {obj['Key']}:", len(reader.pages))
            # for each page, extract the text and send it to the embedding endpoint
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text = page.extract_text()
                # Generate embeddings
                response = requests.post(ovhcloud_embedding_endpoint_url, json={"text": text}, headers={'Authorization': f'Bearer {ovhcloud_ai_endpoint_api_key}'})
                embeddings = response.json()
                data = [(obj['Key'], page_num, text, embeddings)]
                # Insert embeddings into PostgreSQL
                try:
                    execute_values(
                        cur, 
                        "INSERT INTO embeddings (document_name, page_number, text, embedding) VALUES %s", 
                        data
                    )
                    conn.commit()
                except Exception as e:
                    print(f"Error inserting data for {obj['Key']} page {page_num}: {e}")
                    print("INSERT INTO embeddings (document_name, page_number, text, embedding) VALUES %s" % data)
                    conn.rollback()
                # 3 second wait in order now to reach API limit calls
                time.sleep(3)

            # close the file and then delete the temp file
            file.close()
        # delete the temp file temp.pdf
        os.remove('temp.pdf')
# check how many rows are in the embeddings table and print it
cur.execute("SELECT COUNT(*) FROM embeddings")
print("DB now has", cur.fetchone()[0], " embeddings")


Now that we have added all the "Terms & Conditions" documents from the S3 bucket files, let's add a different source of information : the OVHcloud documentation.
The documentation is hosted in this github (https://github.com/ovh/docs), as markdown files. Each page exists in several languages so we will take the english pages : guide.en-ie.md

In [ ]:
# Function to get all 'guide.en-ie.md' files in a directory and its subdirectories
def get_guide_md_files(directory):
    guide_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file == 'guide.en-ie.md':
                guide_files.append(os.path.join(root, file))
    return guide_files

# Function to extract sections from markdown text
def extract_md_sections(md_text):
    sections = []
    current_section = []
    lines = md_text.split('\n')
    
    for line in lines:
        if line.startswith('#'):
            if current_section:
                sections.append('\n'.join(current_section))
                current_section = []
        current_section.append(line)
    
    if current_section:
        sections.append('\n'.join(current_section))
    
    return sections


In [ ]:

# Local directory containing markdown files
local_directory = 'docs/pages/'

# Process 'guide.en-ie.md' files in the local directory
guide_files = get_guide_md_files(local_directory)

for guide_file in guide_files:
    with open(guide_file, 'r', encoding='utf-8') as file:
        md_text = file.read()
    
    sections = extract_md_sections(md_text)
    
    for section_num, section_text in enumerate(sections):
        # Add folder structure at the beginning of the text
        folder_structure = os.path.relpath(guide_file, local_directory)
        text_with_structure = f"{folder_structure}\n\n{section_text}"
        
        # Generate embeddings
        response = requests.post(ovhcloud_embedding_endpoint_url, json={"text": text_with_structure}, headers={'Authorization': f'Bearer {ovhcloud_ai_endpoint_api_key}'})
        embeddings = response.json()
        
        data = [(folder_structure, section_num, text_with_structure, embeddings)]
        # Insert embeddings into PostgreSQL
        try:
            execute_values( 
                cur, 
                "INSERT INTO embeddings (document_name, page_number, text, embedding) VALUES %s", 
                data
            )
            conn.commit()
        except Exception as e:
            print(f"Error inserting data for {guide_file} section {section_num}: {e}")
            conn.rollback()
        
        # 3-second wait to avoid reaching API rate limits
        time.sleep(3)

# Check the number of rows in the embeddings table
cur.execute("SELECT COUNT(*) FROM embeddings")
print("DB now has", cur.fetchone()[0], " embeddings")

We are done adding documents in the DB so we close the connection.

In [ ]:

# Close the connection
cur.close()
conn.close()

Now that we have created the knowledge database, lets create a way to query it.
We will be using a chat interface from gradio. This chat interface needs a fonction that will be called each time a new message is sent by a user.

In [ ]:
def rag_response(message, history):
    print(history)
   # Get the embeddings of the query text
    response = requests.post(ovhcloud_embedding_endpoint_url, json={"text": message}, headers={'Authorization': f'Bearer {ovhcloud_ai_endpoint_api_key}'})
    query_embedding = response.json()

    # Connect to the PostgreSQL db and run a similarity query on this embedding to get the top 5 most similar texts
    with psycopg2.connect(dbname=postgre_db, user=postgre_user, password=postgre_password, host=postgre_host, port=postgre_port, sslmode=sslmode) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT document_name, page_number, embedding <-> %s::vector as distance, text FROM embeddings ORDER BY distance LIMIT 5", (query_embedding,))
            results = cur.fetchall()

    # Build the context string for the LLM model by concatenating the top 5 most relevant texts
    system_prompt = "You are a nice chatbot and you have to answer the user question based on the context provided below and no prior knowledge. If the answer was found in a context document, provide the document name and page number. \n <context>"
    system_prompt += ''.join(f"\n Document: {result[0]}, Page: {result[1]}, Text: {result[3]} \n" for result in results)
    system_prompt += "</context>"

    # Build the history with validation for role
    valid_roles = {"user", "assistant", "system", "tool"}
    messages = [{"role": "system", "content": system_prompt}] + [
        {"role": role, "content": content} for role, content in history if role in valid_roles
    ] + [{"role": "user", "content": message}]

    # Call the LLM model API with the user question and system prompt
    response = requests.post(ovhcloud_llm_endpoint_url, json={"max_tokens": 512, "messages": messages, "model": "Mixtral-8x22B-Instruct-v0.1", "temperature": 0}, headers={"Content-Type": "application/json", "Authorization": f"Bearer {ovhcloud_ai_endpoint_api_key}"})

    # Handle response
    return response.json()["choices"][0]["message"]["content"] if response.status_code == 200 else f"{response.status_code} {response.text}"



Now lets launch the chat interface

In [ ]:
gr.ChatInterface(rag_response).launch(share=True)

You can test the fonction directly in the notebook :

In [ ]:
message="Customer is asking if in our AI Deploy service he can access to docker containers using SSH ?"
history=""
rag_response(message, history)